In [1]:
import os

def list_in_mnist_btm_hash(base_dir):
    sub_dirs = ['testing', 'training']
    file_list = []

    for sub_dir in sub_dirs:
        path = os.path.join(base_dir, sub_dir)
        for i in range(10):
            folder_path = os.path.join(path, str(i))
            if os.path.exists(folder_path):
                file_list.extend(os.listdir(folder_path))

    print(f"총 파일 개수: {len(file_list)}")
    return file_list

base_dir = 'mnist_btm_hash' 
file_list = list_in_mnist_btm_hash(base_dir)
print(len(file_list))

총 파일 개수: 69997
69997


In [2]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import os
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F

class AdvancedCNN_MP(nn.Module):
    def __init__(self):
        super(AdvancedCNN_MP, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, padding=1)  
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, padding=1)  
        self.bn2 = nn.BatchNorm2d(20)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(500, 64)  # 입력 크기를 조정
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [3]:
#model load
model = AdvancedCNN_MP()
model.load_state_dict(torch.load('C:/Users/koo/my_workspace/privacy_term_project/advanced_mnist_cnn_by_ADT_mp.pth'))
model.eval()
model.cuda()

AdvancedCNN_MP(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=500, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [4]:
# 노이즈 마스크 생성 함수
def add_noise_to_image(image, num_pixels, intensity=0.5):
    image = image.clone()
    for _ in range(num_pixels):
        x, y = np.random.randint(0, image.size(1)), np.random.randint(0, image.size(2))
        noise = torch.randn(image.size(0)) * intensity
        image[:, x, y] += noise
    return torch.clamp(image, 0, 1)

In [5]:
# 데이터 로딩 및 폴더 생성
test_dir = 'C:/Users/koo/my_workspace/privacy_term_project/mnist_btm/testing'
output_dir = 'C:/Users/koo/my_workspace/privacy_term_project/random_pertubation_image_using_ADT_model_2'

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
test_dataset = ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [6]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    for i in range(10):
        os.makedirs(os.path.join(output_dir, str(i)))

In [7]:
# 이미지 생성 및 모델 테스트
success_counts = {str(i): 0 for i in range(10)}
total_counts = {str(i): 0 for i in range(10)}

In [8]:

# 노이즈 추가 및 모델 예측 부분
for label in range(10):
    label_dir = f'{output_dir}/{label}'
    if not os.path.exists(label_dir):
        os.makedirs(label_dir)
        
    for i, (data, target) in enumerate(test_loader):
        if target.item() != label or total_counts[str(label)] >= 1000:
            continue

        noisy_image = add_noise_to_image(data[0], num_pixels=5, intensity=0.5)
        noisy_image = noisy_image.unsqueeze(0).cuda()
        prediction = model(noisy_image)
        predicted_label = torch.max(prediction, 1)[1].item()

        if predicted_label != target.item():
            success_counts[str(label)] += 1
            noisy_img = transforms.ToPILImage()(noisy_image.squeeze(0).cpu())  # 배치 차원 제거 및 CPU로 이동
            noisy_img.save(f'{output_dir}/{label}/{label}_noisy_{total_counts[str(label)]}.bmp')

        total_counts[str(label)] += 1


In [15]:

# 성공률 출력
for label in range(10):
    success_rate = success_counts[str(label)] / 1000 * 100
    print(f'Label {label}: Success Rate = {success_rate}%')
    
sum = 0
for label in range(10):
    sum += success_counts[str(label)] 
print(sum/10000*100)

Label 0: Success Rate = 53.5%
Label 1: Success Rate = 0.1%
Label 2: Success Rate = 94.39999999999999%
Label 3: Success Rate = 99.9%
Label 4: Success Rate = 2.0%
Label 5: Success Rate = 89.2%
Label 6: Success Rate = 91.3%
Label 7: Success Rate = 100.0%
Label 8: Success Rate = 46.300000000000004%
Label 9: Success Rate = 55.1%
63.18


In [16]:
s_sum = 0
t_sum = 0
for label in range(10):
    s_sum += success_counts[str(label)] 
    t_sum += total_counts[str(label)] 
print(str(s_sum/t_sum*100) + "%")

64.56161863887185%
